# Úkol 1
V souboru s daty na Slacku máš data o hodnotách finančních indikátorů 100 největších společností obchodovaných na americké burze. Naším cílem je zjistit, které indikátory nejvíce ovlivňují cenu, a vytvořit model, který odhadne cenu akcie na základě hodnot finančních indikátorů.

Jeden z indikátorů je označený jako Y a je poměrem ceny a účetní hodnoty akcie. Hodnota indikátoru je odrazem ceny akcie. Ostatní indikátory jso následující:

běžná likvidita (Current Ratio, CR),
zadluženost (Debt to Assets, DA),
finanční páka (Financial Leverage, FL)
provozní zisková marže (Operating Profit Margin, OPM),
obrat pohledávek (Receivables Turnover, RT),
obrat celkových aktiv (Total Assets Turnover, TAT).
Úkol můžeš odevzdat jako Jupyter notebook.
Nenahrávej prosím datový soubor na GitHub (neobsahuje veřejně dostupná data). Ze stejného důvodu jsou data anonymizovaná, tj. není v nich obsažen konkrétní název firmy.
### Část 1
Vytvoř korelační matici a podívej se, který ukazatel má největší vliv na indikátor Y akcie.
### Část 2
Vytvoř regresní model, který bude mít koeficient Y jako vysvětlovanou proměnnou. Do modelu vlož hodnoty ostatních indikátorů jako vysvětlující proměnné.
S využitím modulu scipy vytvoř regresní model a zobraz si tabulku se souhrnem významů. Podívej se na hodnoty koeficientů a na výsledky testu statistické významnosti koeficientů. Pokud je některý koeficient (nebo více koeficientů) nevýznamný, sestav nový model bez tohoto koeficientů (případně beze všech nevýznamných koeficientů).
Pro všechna data odhadni ukazatel Y s využitím tvého modelu a odhadnuté ceny vlož do původní tabulky s daty. Dále vypočítej rozdíl mezi odhadem koeficientu a jeho skutečnou hodnotou. Najdi akcii, kde je tento rozdíl největší (tj. hledáme akcii, které náš model predikuje výrazně vyšší cenu než jaká je ve skutečnosti, tato akcie je potenciálně na trhu podhodnocená)

# Řešení

In [11]:
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

data = pd.read_csv("df.csv")
data.head()

,Company,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,Sector
0,2Q3DU,1.19,0.20,2.21,0.0770,6.66,0.40,0.43,1.95,5.93,Industrials
1,46JNq,0.29,0.64,10.14,0.2037,20.04,0.36,6.52,7.26,43.48,Communication Services
2,4ISOt,2.93,0.04,1.43,0.3055,6.55,0.72,0.06,7.78,18.38,Communication Services
3,5AWwu,1.03,0.25,3.55,0.0708,146.43,1.80,0.88,6.17,5.15,Consumer Defensive
4,5hbZR,1.20,0.47,0.00,0.1677,30.92,0.93,0.00,0.00,17.06,Consumer Cyclical


Tip: Při tvorbě korelační matice pandas zápasí s nečíselnými sloupci. Jednoduché řešení je vytvoření nové tabulky, ze které vyřadíte sloupečky Company a Sector. Z této tabulky už korelační matice vytvořit půjde.

In [2]:
data_corr = data.drop(columns=["Company", "Sector"])

**Korelace** (*correlation*) –to, jak je závislost silná, můžeme popsat pomocí **korelačního koeficientu** (*correlation coefficient*). Pro jeho hodnoty platí následující:

- Hodnoty blízko +1 znamenají silnou přímou lineární závislost, tj. hodnoty v obou sloupcích rostou současně.
- Hodnoty blízko 0 znamenají lineární nezávislost.
- Hodnoty blízko -1 znamenají silnou nepřímou lineární závislost, tj. jedna hodnota roste a současně druhá klesá.


Hodnotu korelace zjistíme pomocí metody `corr()` pro zvolenou tabulku.

Vytvoř korelační matici a podívej se, který ukazatel má největší vliv na indikátor Y akcie.

In [4]:
data_corr.corr()

,CR,DA,FL,OPM,RT,TAT,DE,Y,PT
CR,1.000000,-0.225917,-0.179232,0.284529,-0.151203,-0.123665,-0.141136,-0.018690,-0.040483
DA,-0.225917,1.000000,0.340320,0.015032,0.206620,-0.083691,0.464786,0.164599,-0.120993
FL,-0.179232,0.340320,1.000000,-0.112430,-0.076335,0.156169,0.968612,0.759323,0.032588
OPM,0.284529,0.015032,-0.112430,1.000000,0.051672,-0.298701,-0.050014,0.051308,0.021409
RT,-0.151203,0.206620,-0.076335,0.051672,1.000000,0.206580,-0.065267,-0.046290,0.097708
TAT,-0.123665,-0.083691,0.156169,-0.298701,0.206580,1.000000,0.094869,0.301222,0.017872
DE,-0.141136,0.464786,0.968612,-0.050014,-0.065267,0.094869,1.000000,0.724604,-0.025348
Y,-0.018690,0.164599,0.759323,0.051308,-0.046290,0.301222,0.724604,1.000000,-0.001363
PT,-0.040483,-0.120993,0.032588,0.021409,0.097708,0.017872,-0.025348,-0.001363,1.000000


Podíváme se na korelační koeficienty pro Y. Nejvyšší hodnotu blížící se 1 vidíme u FL 0.759323 a pak vidíme také hodnotu 0.724604 u DE.

Vytvoř regresní model, který bude mít koeficient Y jako vysvětlovanou proměnnou. Do modelu vlož hodnoty ostatních indikátorů jako vysvětlující proměnné.
S využitím modulu scipy vytvoř regresní model a zobraz si tabulku se souhrnem významů. Podívej se na hodnoty koeficientů a na výsledky testu statistické významnosti koeficientů. Pokud je některý koeficient (nebo více koeficientů) nevýznamný, sestav nový model bez tohoto koeficientů (případně beze všech nevýznamných koeficientů).

In [5]:
formula = "Y ~ CR + DA + FL + OPM + RT + TAT + DE + PT"
mod = smf.ols(formula=formula, data=data_corr)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.664
Model:                            OLS   Adj. R-squared:                  0.635
Method:                 Least Squares   F-statistic:                     22.52
Date:                Tue, 31 Oct 2023   Prob (F-statistic):           1.45e-18
Time:                        22:37:14   Log-Likelihood:                -375.94
No. Observations:                 100   AIC:                             769.9
Df Residuals:                      91   BIC:                             793.3
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.0176      5.347     -2.061      0.042     -21.638      -0.397
CR             1.5014      1.240      1.211      0.229      -0.961       3.964
DA            -5.2206      9.370     -0.557      0.579     -23.834      13.392
FL             3.0104      1.060      2.840      0.006       0.905       5.116
OPM           23.4665      8.076      2.906      0.005       7.425      39.508
RT            -0.0063      0.021     -0.300      0.765      -0.048       0.035
TAT            8.4254      2.351      3.584      0.001       3.756      13.095
DE            -0.7283      2.066     -0.352      0.725      -4.833       3.376
PT            -0.0644      0.096     -0.669      0.505      -0.256       0.127
==============================================================================
Omnibus:                       29.136   Durbin-Watson:                   1.900
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              101.551
Skew:                           0.881   Prob(JB):                     8.88e-23
Kurtosis:                       7.611   Cond. No.                         592.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Podívejme se nejprve na dvě čísla (koeficienty - *coefficients*), která potřebujeme k nakreslení naší funkce:

- V řádku "Intercept" máme hodnotu, která určuje, kde funkce protne se svislou osou.
- V řádcích "CR", "DA" a dalších máme hodnotu, která udává sklon funkce. 
Koeficient determinance je označen jako R-squared. Náš model má koeficient determinance 0.664, dokáže tedy vysvětlit přibližně 66,4 procent variability Y.


Test hypotézy o statistické významnosti koeficientu

Test má následující hypotézy:

- H0: Koeficient je statisticky nevýznamný.
- H1: Koeficient je statisticky významný.

Pokud je p-hodnota testu méně než 0.05, můžeme tedy koeficient označit jako statisticky významný. p-hodnotu testu najdeme ve sloupci `P>|t|`. p-hodnotu máme pro každý koeficient zvlášť. V našem případě platí, že:
Koeficient "Intercept" je 0.042 - je statisticky nevýznamný.
Koeficient "CR" je 0.229 - je statisticky významný.
Koeficient "DA" je 0.579 - je statisticky významný.
Koeficient "FL" je 0.006 - je statisticky nevýznamný.
Koeficient "OPM" je 0.005 - je statisticky nevýznamný.
Koeficinet "RT" je 0.765 - je statisticky významný.
Koeficient "TAT" je 0.001 – je statisticky nevýznamný.
Koeficient "DE" je 0.725 – je statisticky významný.
Koeficient "PT" je 0.505 – je statisticky významný.

Sestavíme nový model bez statisticky nevýznamných koeficientů - tedy bez FL, OPM a TAT.

In [7]:
formula = "Y ~ CR + DA + RT + DE + PT"
mod = smf.ols(formula=formula, data=data_corr)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.569
Model:                            OLS   Adj. R-squared:                  0.546
Method:                 Least Squares   F-statistic:                     24.85
Date:                Tue, 31 Oct 2023   Prob (F-statistic):           7.12e-16
Time:                        22:40:48   Log-Likelihood:                -388.41
No. Observations:                 100   AIC:                             788.8
Df Residuals:                      94   BIC:                             804.5
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      8.8813      3.870      2.295      0.024       1.198      16.564
CR             1.0601      1.281      0.827      0.410      -1.484       3.604
DA           -24.7803      8.782     -2.822      0.006     -42.218      -7.343
RT             0.0206      0.022      0.924      0.358      -0.024       0.065
DE             5.3276      0.493     10.800      0.000       4.348       6.307
PT            -0.0180      0.105     -0.172      0.864      -0.227       0.191
==============================================================================
Omnibus:                       28.782   Durbin-Watson:                   1.914
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              107.262
Skew:                           0.839   Prob(JB):                     5.11e-24
Kurtosis:                       7.788   Cond. No.                         464.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Pro všechna data odhadni ukazatel Y s využitím tvého modelu a odhadnuté ceny vlož do původní tabulky s daty. Odhadnuté ceny vložíme do nového sloupce PricePredicted.

In [12]:
data_corr["PricePredicted"] = res.predict(data_corr)
data_corr.head()

,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,PricePredicted,Difference
0,1.19,0.20,2.21,0.0770,6.66,0.40,0.43,1.95,5.93,7.508028,5.558028
1,0.29,0.64,10.14,0.2037,20.04,0.36,6.52,7.26,43.48,27.694182,20.434182
2,2.93,0.04,1.43,0.3055,6.55,0.72,0.06,7.78,18.38,11.119264,3.339264
3,1.03,0.25,3.55,0.0708,146.43,1.80,0.88,6.17,5.15,11.395772,5.225772
4,1.20,0.47,0.00,0.1677,30.92,0.93,0.00,0.00,17.06,-1.163072,-1.163072


Dále vypočítej rozdíl mezi odhadem koeficientu a jeho skutečnou hodnotou. Najdi akcii, kde je tento rozdíl největší (tj. hledáme akcii, které náš model predikuje výrazně vyšší cenu než jaká je ve skutečnosti, tato akcie je potenciálně na trhu podhodnocená).

In [13]:
data_corr["Difference"] = data_corr["PricePredicted"] - data_corr["Y"]
data_corr.head()

,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,PricePredicted,Difference
0,1.19,0.20,2.21,0.0770,6.66,0.40,0.43,1.95,5.93,7.508028,5.558028
1,0.29,0.64,10.14,0.2037,20.04,0.36,6.52,7.26,43.48,27.694182,20.434182
2,2.93,0.04,1.43,0.3055,6.55,0.72,0.06,7.78,18.38,11.119264,3.339264
3,1.03,0.25,3.55,0.0708,146.43,1.80,0.88,6.17,5.15,11.395772,5.225772
4,1.20,0.47,0.00,0.1677,30.92,0.93,0.00,0.00,17.06,-1.163072,-1.163072


Najdeme největší rozdíl.

In [16]:
max_difference = data_corr["Difference"].max()
max_difference

39.52221610987559

Najdeme řádek, na kterém je tento největší rozdíl.

In [17]:
row_with_max_difference = data_corr["Difference"].idxmax()
row_with_max_difference

98

Najdeme název společnosti z původní tabulky, která je na daném řádku, tedy jde o akcie této společnosti.

In [18]:
value_in_column_company = data.at[98, "Company"]
value_in_column_company

'YX515'

Největší rozdíl mezi odhadem koeficientu a jeho skutečnou hodnotou – akcie, kde je tento rozdíl největší (tj. hledáme akcii, které náš model predikuje výrazně vyšší cenu než jaká je ve skutečnosti, tato akcie je potenciálně na trhu podhodnocená)
Výsledek: Jde o akcii společnosti YX515 a rozdíl je 39.52221610987559.

Bonus 1: Sestav model s využitím robustní regrese. Opět proveď vyřazení koeficinetů, které nejsou statisticky významné, a sestav model pouze s významnými koeficienty. Vlož odhady cen do původních dat a opět najdi potenciálně nejvíce podhodnocenou akcii. Jde v případě robustní regese o stejnou akcii, nebo se akcie liší?

Robustní regrese

In [19]:
formula = "Y ~ CR + DA + FL + OPM + RT + TAT + DE + PT"
mod = smf.rlm(formula=formula, data=data_corr)
res2 = mod.fit()
res2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                    Robust linear Model Regression Results                    
==============================================================================
Dep. Variable:                      Y   No. Observations:                  100
Model:                            RLM   Df Residuals:                       91
Method:                          IRLS   Df Model:                            8
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Tue, 31 Oct 2023                                         
Time:                        23:58:34                                         
No. Iterations:                    37                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -10.4961      3.453     -3.039      0.002     -17.265      -3.728
CR             2.0316      0.801      2.537      0.011       0.462       3.601
DA            -0.4762      6.052     -0.079      0.937     -12.338      11.386
FL             3.6694      0.685      5.360      0.000       2.328       5.011
OPM           16.3810      5.216      3.140      0.002       6.158      26.604
RT            -0.0029      0.014     -0.216      0.829      -0.029       0.024
TAT            5.2670      1.518      3.469      0.001       2.291       8.243
DE            -2.5507      1.335     -1.911      0.056      -5.166       0.065
PT            -0.0878      0.062     -1.412      0.158      -0.210       0.034
==============================================================================

If the model instance has been used for another fit with different fit parameters, then the fit options might not be the correct ones anymore .
"""

Vyřadíme statisticky nevýznamné koeficienty: tedy CR (hodnota 0.011), FL (hodnota 0.000), OPM (hodnota 0.002), TAT (hodnota 0.001)

In [20]:
formula = "Y ~ DA + RT + DE + PT"
mod = smf.rlm(formula=formula, data=data_corr)
res2 = mod.fit()
res2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                    Robust linear Model Regression Results                    
==============================================================================
Dep. Variable:                      Y   No. Observations:                  100
Model:                            RLM   Df Residuals:                       95
Method:                          IRLS   Df Model:                            4
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Tue, 31 Oct 2023                                         
Time:                        23:59:16                                         
No. Iterations:                    50                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      8.3422      1.639      5.089      0.000       5.129      11.555
DA           -18.1989      4.954     -3.674      0.000     -27.908      -8.489
RT             0.0097      0.013      0.764      0.445      -0.015       0.035
DE             4.3945      0.281     15.635      0.000       3.844       4.945
PT             0.0158      0.060      0.264      0.791      -0.102       0.133
==============================================================================

If the model instance has been used for another fit with different fit parameters, then the fit options might not be the correct ones anymore .
"""

Pro všechna data odhadneme ukazatel Y s využitím našeho modelu a odhadnuté ceny vložíme do původní tabulky s daty. Odhadnuté ceny vložíme do nového sloupce PricePredictedRob.

In [21]:
data_corr["PricePredictedRob"] = res2.predict(data_corr)
data_corr.head()

,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,PricePredicted,Difference,PricePredictedRob
0,1.19,0.20,2.21,0.0770,6.66,0.40,0.43,1.95,5.93,7.508028,5.558028,6.750420
1,0.29,0.64,10.14,0.2037,20.04,0.36,6.52,7.26,43.48,27.694182,20.434182,26.229492
2,2.93,0.04,1.43,0.3055,6.55,0.72,0.06,7.78,18.38,11.119264,3.339264,8.232365
3,1.03,0.25,3.55,0.0708,146.43,1.80,0.88,6.17,5.15,11.395772,5.225772,9.158699
4,1.20,0.47,0.00,0.1677,30.92,0.93,0.00,0.00,17.06,-1.163072,-1.163072,0.358182


Vypočítáme rozdíl.

In [22]:
data_corr["DifferenceRob"] = data_corr["PricePredictedRob"] - data_corr["Y"]
data_corr.head()
data_corr

,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,PricePredicted,Difference,PricePredictedRob,DifferenceRob
0,1.19,0.20,2.21,0.0770,6.66,0.40,0.43,1.95,5.93,7.508028,5.558028,6.750420,4.800420
1,0.29,0.64,10.14,0.2037,20.04,0.36,6.52,7.26,43.48,27.694182,20.434182,26.229492,18.969492
2,2.93,0.04,1.43,0.3055,6.55,0.72,0.06,7.78,18.38,11.119264,3.339264,8.232365,0.452365
3,1.03,0.25,3.55,0.0708,146.43,1.80,0.88,6.17,5.15,11.395772,5.225772,9.158699,2.988699
4,1.20,0.47,0.00,0.1677,30.92,0.93,0.00,0.00,17.06,-1.163072,-1.163072,0.358182,0.358182
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.50,0.37,2.33,0.2557,4.92,0.41,0.85,5.18,6.83,5.809450,0.629450,5.499701,0.319701
96,0.70,0.27,2.57,0.2363,16.11,0.64,0.69,7.40,2.70,6.892482,-0.507518,6.659399,-0.740601
97,1.49,0.12,2.36,0.2576,8.64,0.51,0.28,5.99,7.59,9.020248,3.030248,7.592612,1.602612
98,2.30,0.64,25.03,0.3758,7.48,0.31,16.08,41.57,10.54,81.092216,39.522216,67.597631,26.027631


Vypočítáme největší rozdíl.

In [23]:
max_difference_rob = data_corr['DifferenceRob'].max()
max_difference_rob

26.0276308091831

Najdeme řádek s největším rozdílem.

In [24]:
row_with_max_difference_rob = data_corr['DifferenceRob'].idxmax()
row_with_max_difference_rob

98

Najdeme, o kterou společnost se jedná.

In [25]:
value_in_column_company = data.at[98, 'Company']
value_in_column_company

'YX515'

Robustní regrese výsledek: Největší rozdíl mezi odhadem koeficientu a jeho skutečnou hodnotou – akcie, kde je tento rozdíl největší (tj. hledáme akcii, které náš model predikuje výrazně vyšší cenu než jaká je ve skutečnosti, tato akcie je potenciálně na trhu podhodnocená), jde o stejný řádek, tedy akcii stejné společnosti – YX515, ale rozdíl vyšel trochu jiný: je to 26.0276308091831.